In [5]:
import sys; sys.path.append('..'); from wwa import *

Xn, Xx, Yn, Yx = [-67,-47,-27,-7]
xn, xx, yn, yx = [-60,-54,-23,-15]
mapproj = cartopy.crs.PlateCarree()

# Pantanal shapefile
Region of interest is the flood plain of the upper Paraguay river - use hydrobasin to refine area, then cut to within Brazil (could also include part of Bolivia?) & possibly use elevation to further restrict to just the floodplain

In [21]:
# clip shapefile to just Brazil
sf_hybas = gpd.read_file("sf/hybas04").loc[[83]]

gdf_br = regionmask.defined_regions.natural_earth_v5_0_0.countries_50.to_geodataframe().set_crs(epsg = 4326).set_index("abbrevs").loc[["BR", "BO"]]
sf_hybas_pan = main_polygon(sf_hybas.clip(gdf_br)).clip([-60.75,-23,-50,-10])

sf_hybas_pan.to_file("sf/hybas_br-bo")

/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/geopandas/geodataframe.py:2233: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...
Right CRS: EPSG:4326

  return geopandas.clip(self, mask=mask, keep_geom_type=keep_geom_type)
/rds/general/user/cb2714/home/wwa/24-06-20_brazil-wildfires/../wwa.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = gdf.loc[gdf.geometry.area == gdf.geometry.area.max()]


In [6]:
# Pantanal biome shapefile
# ! cd data; wget http://geoftp.ibge.gov.br/informacoes_ambientais/estudos_ambientais/biomas/vetores/Biomas_250mil.zip; unzip data/Biomas_250mil.zip -d sf/biomas

In [3]:
# shapefile of Brazilian Pantanal from https://brasil.mapbiomas.org/en/mapas-de-referencia/
sf_biomas = gpd.read_file("sf/biomas").loc[[5]].to_crs(epsg = 4326)
sf_biomas.to_file("sf/pantanal")

## Compile & filter active fire data

In [2]:
# MODIS active fire pixels
# ! cd data; wget https://firms.modaps.eosdis.nasa.gov/data/download/DL_FIRE_M-C61_489623.zip; unzip data/DL_FIRE_M-C61_489623.zip -d data

In [9]:
# load all active fire data
df = pd.concat([pd.read_csv(fnm, parse_dates = ["acq_date"], index_col = "acq_date") for fnm in glob.glob("data/*M-C61*.csv")])

# filter out low-confidence points
df = df.loc[(df.confidence >= 80), ["latitude", "longitude"]]

df.to_csv("data/modis_conf-80.csv")

# ERA5 FWI download via API

In [ ]:
xn, xx, yn, yx = [-60,-54,-23,-15]

In [ ]:
import cdsapi

c = cdsapi.Client()

for y in range(2023, 1949,-1):
    
    c.retrieve(
        'cems-fire-historical-v1',
        {
            'product_type': 'reanalysis',
            'variable': [
                'build_up_index', 'drought_code', 'duff_moisture_code',
                'fine_fuel_moisture_code', 'initial_fire_spread_index',
                'keetch_byram_drought_index',
            ],
            'dataset_type': 'consolidated_dataset',
            'system_version': '4_1',
            'year': str(y),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'grid': '0.25/0.25',
            'format': 'netcdf',
            'area': [
                yx, xn, yn, xx,
            ],
        },
        'data/era5-fire/fwi-subindices_cons_'+str(y)+'.nc')

2024-07-16 10:52:32,295 INFO Welcome to the CDS
2024-07-16 10:52:32,295 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/cems-fire-historical-v1
2024-07-16 10:52:32,384 INFO Request is queued


In [ ]:
import cdsapi

c = cdsapi.Client()

for y in range(2023, 2025):
    
    c.retrieve(
        'cems-fire-historical-v1',
        {
            'product_type': 'reanalysis',
            'variable': [
                'build_up_index', 'drought_code', 'duff_moisture_code',
                'fine_fuel_moisture_code', 'initial_fire_spread_index',
                'keetch_byram_drought_index',
            ],
            'dataset_type': 'intermediate_dataset',
            'system_version': '4_1',
            'year': str(y),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'grid': '0.25/0.25',
            'format': 'netcdf',
            'area': [
                yx, xn, yn, xx,
            ],
        },
        'data/era5-fire/fwi-subindices_int_'+str(y)+'.nc')

2024-07-16 10:52:32,295 INFO Welcome to the CDS
2024-07-16 10:52:32,295 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/cems-fire-historical-v1
2024-07-16 10:52:32,384 INFO Request is queued


In [1]:
# KBDI from ERA5

! cd data; wget https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data1/adaptor.mars.external-1721119058.7953396-3576-7-659d00ad-cc77-4c91-a11a-23e1d2bc86fd.nc

--2024-07-16 09:42:05--  https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data1/adaptor.mars.external-1721119058.7953396-3576-7-659d00ad-cc77-4c91-a11a-23e1d2bc86fd.nc
Resolving download-0005-clone.copernicus-climate.eu (download-0005-clone.copernicus-climate.eu)... 2a0c:5bc0:4c:ff::889c:9986, 136.156.153.134
Connecting to download-0005-clone.copernicus-climate.eu (download-0005-clone.copernicus-climate.eu)|2a0c:5bc0:4c:ff::889c:9986|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221138 (216K) [application/x-netcdf]
Saving to: ‘adaptor.mars.external-1721119058.7953396-3576-7-659d00ad-cc77-4c91-a11a-23e1d2bc86fd.nc’

adaptor.mars.extern 100%[===================>] 215.96K  --.-KB/s    in 0.1s    

2024-07-16 09:42:06 (1.84 MB/s) - ‘adaptor.mars.external-1721119058.7953396-3576-7-659d00ad-cc77-4c91-a11a-23e1d2bc86fd.nc’ saved [221138/221138]



# Get DSR

In [8]:
# Compile FWI data into a single file
fl = sorted(glob.glob("../../00_WWA_project_folder/live/data/era5_fire/*consolidated*.nc"))

fwi_cns = [wrap_lon(xr.open_dataset(fnm)).fwinx.sel(longitude = slice(xn, xx), latitude = slice(yn, yx)).reset_coords(drop = True) for fnm in fl]

int_start = (fwi_cns[-1].time[-1]+1)

fwi_int = wrap_lon(xr.open_dataset("../../00_WWA_project_folder/live/data/era5_fire/era5_025_fwi_intermediate.nc"))
fwi_int = fwi_int.fwinx.sel(longitude = slice(xn, xx), latitude = slice(yn, yx), time = slice(int_start, None)).reset_coords(drop = True)

fwi = xr.concat(fwi_cns + [fwi_int], "time")
fwi = fwi.assign_attrs(note = "Intermediate data after "+str(fwi_int.time.min().values)[:10])

fwi.to_netcdf("data/era5_fwi_"+"_".join([str(d) for d in [xn,xx,yn,yx]])+".nc")

In [5]:
dsr = (0.0272 * (fwi ** 1.71)).rename("dsr")

In [6]:
dsr.to_netcdf("data/era5_dsr.nc")